In [3]:
from genetic_algorithm import *
import re
import json
import numpy as np
from numpy.random import default_rng
from collections import Counter
import copy
import matplotlib.pyplot as plt
from itertools import groupby

## hexadecimal generator

In [51]:
nr_of_input = 3
nr_of_actions = 3
nr_of_inner = 3
nr_of_genes = 16
nr_individuals = 20

individuals = generate_initial_genomes_for_population(nr_individuals, nr_of_genes, nr_of_input, nr_of_actions, nr_of_inner)

In [52]:
## world size
world_size_x = 128
world_size_y = 128

rng = default_rng()
x = rng.choice(world_size_x, size=nr_individuals, replace=False)
y = rng.choice(world_size_y, size=nr_individuals, replace=False)


In [53]:
individuals_sum_dup = sum_duplicated_neurons(individuals)

## initial brain and position generator

In [54]:
initial_dic_of_output_neurons = {}

## sum duplicates
individuals_sum_dup = sum_duplicated_neurons(individuals)

## brain generator
for individual in individuals_sum_dup:
    
    ## init brain neuron generator
    print(individual)
    ## preprocessing
    edges = individuals_sum_dup[individual]
    edges = [tuple(edges[i]) for i in edges]
    edges_no_weight = [i[:-1] for i in edges]
    
    G = nx.MultiDiGraph()
    G.add_edges_from(edges)
    
    out_list = list(set([i[1] for i in edges if 'out' in i[1]]))
    init_list = list(set([i[0] for i in edges if 'in' in i[0]]))
    print(init_list)
    # generate dict of paths
    dic_of_paths = generate_dict_of_paths(out_list, init_list, G)

    lis = filtered_neurons_paths(dic_of_paths)
    
    dic = append_weight_to_neurons_in_path(lis, edges_no_weight, edges)

    initial_dic_of_output_neurons[individual] = dic
    print(dic)    
        

#     print(dic)
    print()

0
['in0', 'in1']
{'mid0': {'mid2': -2.7630703494582636, 'in0': 3.8327483595299863, 'mid0': 2.7793071875476882}, 'mid1': -0.7302600376663501, 'mid2': {'mid1': 2.3455516557302, 'mid2': 2.635373111551961}, 'out2': {'mid0': -0.04333892873493057, 'mid2': 0.7360292995574546}}
{'out1': {'out1': -0.9993175754035977}, 'out2': {'out2': 0.9908517382709082}, 'out0': {'out0': -0.09430069463663865}}

1
['in0', 'in1', 'in2']
{'out0': {'out0': -0.9999860429919202}, 'out1': {'out1': 0.7081702535913587}}

2
['in0', 'in1', 'in2']
{'out1': {'out1': -0.9130485400417707}, 'out0': {'out0': 0.9999947571432442}}

3
['in0', 'in1', 'in2']
{'mid0': {'mid2': 3.014680299099649}, 'mid1': 0.7318514403383639, 'mid2': {'mid1': -3.2010987334045473, 'mid2': -2.885762246299405, 'in1': -3.5873645658477034, 'in0': -2.603265679841294}, 'out2': {'mid0': -0.6823134442240195, 'mid2': 3.2311307797955133}}
{'out1': {'out1': 0.9693767017691123}, 'out2': {'out2': 0.9877353444794669}, 'out0': {'out0': -0.17045103720481336}}

4
['in0

In [37]:
dic = {'mid0': {'mid0': 1.8478254234701663, 'in1': 1.7749427743018464, 'mid2': -3.906241416145277}, 'mid1': -0.9999224478418683, 'mid2': {'in1': -0.07153975278498398, 'mid1': -0.792797192125744}, 'out1': {'in1': -3.3650541736609187, 'mid0': 0.92733099343812, 'mid2': -3.8355562337860523}}
dic

{'mid0': {'mid0': 1.8478254234701663,
  'in1': 1.7749427743018464,
  'mid2': -3.906241416145277},
 'mid1': -0.9999224478418683,
 'mid2': {'in1': -0.07153975278498398, 'mid1': -0.792797192125744},
 'out1': {'in1': -3.3650541736609187,
  'mid0': 0.92733099343812,
  'mid2': -3.8355562337860523}}

In [42]:
filter_list = ['in0', 'in2', 'in1']
def sum_weights(dic, filter_list):
    for key in dic:
        if 'mid' in key and isinstance(dic[key], dict):
            mid_to_update = set(dic[key]).difference(set(filter_list + [key]))
            k = 0
            for mid_key in mid_to_update:
                if isinstance(dic[mid_key], float):
                    dic[key][mid_key] = np.tanh(sum([dic[key][mid_key], dic[mid_key]]))
                    k+=1
            if k == len(mid_to_update):
                dic[key] = np.tanh(sum(dic[key].values()))
                sum_weights(dic, filter_list)
                
        elif 'out' in key and isinstance(dic[key], dict):
            for mid_key in dic[key]:
                if 'mid' in mid_key and isinstance(dic[mid_key], float):
                    dic[key][mid_key] = np.tanh(sum([dic[key][mid_key], dic[mid_key]]))
            print(dic[key])
            dic[key] = np.tanh(sum(dic[key].values()))    

sum_weights(dic, filter_list)
dic


{'in1': -3.3650541736609187, 'mid0': 0.9576568054653422, 'mid2': -0.9997997269485884}


{'mid0': 0.9895169151581463,
 'mid1': -0.9999224478418683,
 'mid2': -0.7688817184613416,
 'out1': -0.9978066917654894}

In [38]:
dic

{'mid0': {'mid0': 1.8478254234701663,
  'in1': 1.7749427743018464,
  'mid2': -3.906241416145277},
 'mid1': -0.9999224478418683,
 'mid2': {'in1': -0.07153975278498398, 'mid1': -0.792797192125744},
 'out1': {'in1': -3.3650541736609187,
  'mid0': 0.92733099343812,
  'mid2': -3.8355562337860523}}

In [44]:
mid0 = np.tanh(sum([0.9895169151581463, 0.92733099343812]))
mid2 = np.tanh(sum([-3.8355562337860523, -0.7688817184613416]))
in1 = -3.3650541736609187
in1, mid0, mid2

(-3.3650541736609187, 0.9576568054653422, -0.9997997269485884)

In [45]:
np.tanh(sum([mid0, mid2, in1]))

-0.9978066917654894

In [ ]:
dic = {'mid0': {'mid0': 1.8478254234701663, 'in1': 1.7749427743018464, 'mid2': -3.906241416145277}, 'mid1': -0.9999224478418683, 'mid2': {'in1': -0.07153975278498398, 'mid1': -0.792797192125744}, 'out1': {'in1': -3.3650541736609187, 'mid0': 0.92733099343812, 'mid2': -3.8355562337860523}}

mid_value = {}

for key in dic:
#     print(key)
    if 'mid' in key and isinstance(dic[key], float):
        mid_value[key] = dic[key]

    elif 'mid' in key and isinstance(dic[key], dict):
        for sub_key in dic[key]:
            if sub_key in mid_value:
                dic[key][sub_key] += mid_value[sub_key]

for key in mid_value:
    del dic[key]

# dic_list = {}
# for key in dic:
#     lista = []
#     for sub_key in dic[key]:
#         if sub_key in dic:
#             lista.append(sub_key)
#     dic_list[key] = lista
    
# def sum_weights(dic):
#     for key in dic_list:
#         if dic_list:
#             sum_weights

            
    

dic

In [ ]:
any(dic['mid0']) in dic

In [ ]:
 np.tanh(sum([-0.9999224478418683, 1.8478254234701663, -0.9530708314545707, 1.7749427743018464]))

In [ ]:
out_to_weight(dic)

In [ ]:
len({'in1': -0.07153975278498398, 'mid1': -0.792797192125744})

In [ ]:
dic

In [ ]:
initial_dic_of_output_neurons = {}

## sum duplicates
individuals_sum_dup = sum_duplicated_neurons(individuals)

## remove self loops
individuals_sum_dup_no_self_loop = remove_self_loop(individuals_sum_dup)

## brain generator
for individual in individuals_sum_dup_no_self_loop:
    
    ## init brain neuron generator
    
    ## preprocessing
    edges = individuals_sum_dup_no_self_loop[individual]
    edges = [tuple(edges[i]) for i in edges]

    remove_mid_with_no_predecessor(edges) 
    dic = weight_sum_preprocessing(edges)

    ## calculate weight sum
    mid_to_weight(dic)
    out_to_weight(dic)
       
    dic = {k:v for k, v in dic.items() if 'out' in k}    
    
    initial_dic_of_output_neurons[individual] = {}
    initial_dic_of_output_neurons[individual]['output'] = dic
    
    ## position
#     initial_dic_of_output_neurons[individual]['position'] = {}
#     initial_dic_of_output_neurons[individual]['position']['x'] = x[individual]
#     initial_dic_of_output_neurons[individual]['position']['y'] = y[individual] 
    
    ## position history
    initial_dic_of_output_neurons[individual]['history_position'] = [(x[individual], y[individual])]

In [ ]:
initial_dic_of_output_neurons

In [ ]:
a = [(3, 0),(4,1),(5,2)]

In [ ]:
def slope_intercept(x1,y1,x2,y2):
    a = (y2 - y1) / (x2 - x1)
    b = y1 - a * x1     
    return a,b

print(slope_intercept(3,0,2,1))

In [ ]:
a

In [ ]:
if a[-2][0] == a[-1][0]:
    x = a[-1][0]
elif a[-2][0] > a[-1][0]:
    x = a[-2][0] + 1
elif a[-2][0] < a[-1][0]:
    x = a[-2][0] - 1
print(x)

In [ ]:
a[-2][0]